In [1]:
import torch
import requests
import numpy as np

from PIL import Image

from transformers import (
    AutoProcessor,
    RTDetrForObjectDetection,
    VitPoseForPoseEstimation,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

url = "http://images.cocodataset.org/val2017/000000000139.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# ------------------------------------------------------------------------
# Stage 1. Detect humans on the image
# ------------------------------------------------------------------------

# You can choose detector by your choice
person_image_processor = AutoProcessor.from_pretrained("PekingU/rtdetr_r50vd_coco_o365")
person_model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd_coco_o365", device_map=device)

inputs = person_image_processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = person_model(**inputs)

results = person_image_processor.post_process_object_detection(
    outputs, target_sizes=torch.tensor([(image.height, image.width)]), threshold=0.3
)
result = results[0]  # take first image results

# Human label refers 0 index in COCO dataset
person_boxes = result["boxes"][result["labels"] == 0]
person_boxes = person_boxes.cpu().numpy()

# Convert boxes from VOC (x1, y1, x2, y2) to COCO (x1, y1, w, h) format
person_boxes[:, 2] = person_boxes[:, 2] - person_boxes[:, 0]
person_boxes[:, 3] = person_boxes[:, 3] - person_boxes[:, 1]

# ------------------------------------------------------------------------
# Stage 2. Detect keypoints for each person found
# ------------------------------------------------------------------------

image_processor = AutoProcessor.from_pretrained("usyd-community/vitpose-base-simple")
model = VitPoseForPoseEstimation.from_pretrained("usyd-community/vitpose-base-simple", device_map=device)

inputs = image_processor(image, boxes=[person_boxes], return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

pose_results = image_processor.post_process_pose_estimation(outputs, boxes=[person_boxes], threshold=0.3)
image_pose_result = pose_results[0]  # results for first image

for i, person_pose in enumerate(image_pose_result):
    print(f"Person #{i}")
    for keypoint, label, score in zip(
        person_pose["keypoints"], person_pose["labels"], person_pose["scores"]
    ):
        keypoint_name = model.config.id2label[label.item()]
        x, y = keypoint
        print(f" - {keypoint_name}: x={x.item():.2f}, y={y.item():.2f}, score={score.item():.2f}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

c:\Users\user\miniconda3\envs\lang_311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--PekingU--rtdetr_r50vd_coco_o365. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


preprocessor_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

c:\Users\user\miniconda3\envs\lang_311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--usyd-community--vitpose-base-simple. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/344M [00:00<?, ?B/s]

Person #0
 - Nose: x=428.72, y=170.61, score=0.92
 - L_Eye: x=429.48, y=167.83, score=0.90
 - R_Eye: x=428.74, y=168.17, score=0.79
 - L_Ear: x=433.88, y=167.35, score=0.94
 - R_Ear: x=441.09, y=166.86, score=0.90
 - L_Shoulder: x=440.02, y=177.15, score=0.93
 - R_Shoulder: x=446.28, y=178.39, score=0.74
 - L_Elbow: x=436.88, y=197.90, score=0.92
 - R_Elbow: x=433.36, y=201.21, score=0.54
 - L_Wrist: x=431.45, y=218.66, score=0.88
 - R_Wrist: x=420.10, y=212.79, score=0.96
 - L_Hip: x=444.81, y=224.15, score=0.81
 - R_Hip: x=452.34, y=223.91, score=0.82
 - L_Knee: x=442.23, y=256.02, score=0.83
 - R_Knee: x=451.12, y=255.19, score=0.82
 - L_Ankle: x=443.18, y=288.16, score=0.60
 - R_Ankle: x=456.03, y=285.75, score=0.82
Person #1
 - Nose: x=398.12, y=181.71, score=0.87
 - L_Eye: x=398.45, y=179.73, score=0.82
 - R_Eye: x=396.07, y=179.45, score=0.90
 - R_Ear: x=388.85, y=180.22, score=0.88
 - L_Shoulder: x=397.24, y=194.16, score=0.76
 - R_Shoulder: x=384.60, y=190.74, score=0.64
 - L_

In [6]:
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import AutoProcessor, VitPoseForPoseEstimation

# 📌 1. GPU 설정 (사용 가능하면 CUDA, 아니면 CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 📌 2. ViTPose 모델 및 프로세서 로드
model_name = "usyd-community/vitpose-base-simple"
pose_processor = AutoProcessor.from_pretrained(model_name)
pose_model = VitPoseForPoseEstimation.from_pretrained(model_name).to(device)
pose_model.eval()

# 📌 3. 동영상 로드
video_path = "data/sample1_360.mp4"  # 입력 동영상 파일
output_path = "data/sample1_360_out.mp4"  # 결과 저장 파일

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# COCO 데이터셋 기준의 관절 연결 정보
skeleton = [
    (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 (오른쪽, 왼쪽)
    (11, 13), (13, 15), (12, 14), (14, 16),  # 다리 (오른쪽, 왼쪽)
    (5, 6), (11, 12), (5, 11), (6, 12)  # 몸통 연결
]

# 📌 4. 동영상 프레임별 처리 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 📌 4-1. OpenCV 프레임을 PIL 이미지로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image)

    # 📌 4-2. ViTPose 모델 입력 전처리 (boxes=[] 추가)
    inputs_pose = pose_processor(
        images=image_pil, 
        boxes=[[0, 0, frame_width, frame_height]],  # 전체 이미지를 대상으로 포즈 감지
        return_tensors="pt"
    ).to(device)


    with torch.no_grad():
        outputs_pose = pose_model(**inputs_pose)

    # 포즈 데이터 후처리
    pose_results = pose_processor.post_process_pose_estimation(outputs_pose, threshold=0.3)
    image_pose_result = pose_results[0]  # 첫 번째 이미지의 결과

    # 📌 4-3. 포즈 시각화
    for person_pose in image_pose_result:
        for keypoint, label, score in zip(person_pose["keypoints"], person_pose["labels"], person_pose["scores"]):
            if score.item() > 0.5:  # 신뢰도 50% 이상인 경우만 시각화
                x, y = int(keypoint[0].item()), int(keypoint[1].item())
                cv2.circle(frame, (x, y), 4, (0, 255, 0), -1)  # 관절 점 시각화

    # 📌 4-4. 관절 연결선 (스켈레톤) 시각화
    for pt1, pt2 in skeleton:
        if len(image_pose_result) > 0:
            keypoints = image_pose_result[0]["keypoints"]
            if keypoints[pt1][2] > 0.5 and keypoints[pt2][2] > 0.5:
                x1, y1 = int(keypoints[pt1][0]), int(keypoints[pt1][1])
                x2, y2 = int(keypoints[pt2][0]), int(keypoints[pt2][1])
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 관절 연결선 시각화

    # 📌 4-5. 결과 프레임 저장
    out.write(frame)

    # 📌 4-6. 화면 출력 (실시간 보기, 'q' 키로 종료 가능)
    cv2.imshow('Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 📌 5. 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()


TypeError: 'int' object is not subscriptable